# Malloovia - Data for GECON 2017

This notebook shows how to obtain the data in the paper "Cost Minimization of Virtual Machine Allocation in Public Clouds Considering Multiple Applications" presented in [GECON2017](http://2017.gecon-conference.org/).

In order to run this notebook, you will need to install [Malloovia](https://github.com/asi-uniovi/malloovia). See the [installation section](https://malloovia.readthedocs.io/en/latest/install.html) in the documentation.

The data consists of three data sets, each one in its own directory:

- **problems**: the problems with the workload and the infrastructure.
- **sol_phase_i**: the solutions for Phase I.
- **sol_phase_ii**: the solutions for Phase II.

In each data set, there is a file for each quantization amount, as shown in table 2 in the paper. The files are distributed following a [YAML format defined in Malloovia](https://malloovia.readthedocs.io/en/latest/yaml.html).

Notice that if you run all the cells in this notebook, you will overwrite the solutions and the results (such as the time to solve that is part of the statistics) will change.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("bmh")
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import FloatProgress
import pulp

import malloovia

# Loading data

In [2]:
filenames = ['p0_gcd_increments_None.yaml',
             'p1_gcd_increments_1.yaml',
             'p2_gcd_increments_3.yaml',
             'p3_gcd_increments_5.yaml',
             'p4_gcd_increments_7.yaml',
             'p5_gcd_increments_10.yaml',
             'p6_gcd_increments_15.yaml']

experiments = {} # One per quantization amount
for filename in filenames:
    q_amount = int(filename[1])
    experiments[q_amount] = malloovia.read_problems_from_yaml(
        'problems/' + filename)

# Solution
## Phase I

In [3]:
f = FloatProgress(min=0, max=len(experiments)-1)
display(f)

phase_i_solutions = {}
for i, exp in experiments.items():
    f.description = 'Solving phase I for quantization amount ' + str(i)
    solver = pulp.COIN(maxSeconds=20*60, msg=1, fracGap=0.01)
    phase_i_solutions[i] = malloovia.PhaseI(exp['Problem0']).solve(solver=solver)
    f.value += 1

Notice that the next cell will overwrite the solution files provided.

In [4]:
for q_amount, sol in phase_i_solutions.items():
    s_yaml = malloovia.util.solutions_to_yaml([sol])
    filename = 'sol_phase_i/p{}_phase_i_solution.yaml'.format(q_amount)
    with open(filename, 'w') as f:
        f.write(s_yaml)

## Phase II

In [5]:
f = FloatProgress(min=0, max=len(experiments)-1)
display(f)

phase_ii_solutions = {}
for i, exp in experiments.items():
    f.description = 'Solving phase II for quantization amount ' + str(i)
    phase_ii = malloovia.PhaseII(
        problem = exp['Problem1'],
        phase_i_solution = phase_i_solutions[i],
        reuse_rsv = True)
    phase_ii_solutions[i] = phase_ii.solve_period()
    f.value += 1

Notice that the next cell will overwrite the solution files provided.

In [6]:
for q_amount, sol in phase_ii_solutions.items():
    sol_yaml = malloovia.util.solutions_to_yaml([sol])
    
    filename = 'sol_phase_ii/p{}_phase_ii_solution.yaml'.format(q_amount)
    with open(filename, 'w') as f:
        f.write(sol_yaml)